In [6]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [7]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [119]:
# 비디오 파일 이름
input_file = "pitch_videos/yong11.mp4" # 여기에 비디오 경로 넣으면 됩니다!

# 비디오 파일의 경우 이것을 사용하세요:
cap = cv2.VideoCapture(input_file)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

In [120]:
# 출력 파일 경로 및 이름 설정
output_file = os.path.splitext(input_file)[0] + "_dotted.mp4"
out = cv2.VideoWriter(output_file,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

In [121]:
# 팔꿈치 관절 좌표화 코드
# left_elbow_coordinates = [] # 좌완투수
right_elbow_coordinates = [] # 우완투수, 위 리스트와 둘 중 하나는 주석처리해야 합니다.

with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("비디오 파일을 처리하는데 실패했습니다.")
            break

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        
        image_height, image_width, _ = image.shape

        if results.pose_landmarks:
            # left_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            # cv2.circle(image, (int(left_elbow.x * image_width), int(left_elbow.y * image_height)), 5, (0, 255, 0), -1)
            # left_elbow_coordinates.append((left_elbow.x, left_elbow.y))

            right_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            cv2.circle(image, (int(right_elbow.x * image_width), int(right_elbow.y * image_height)), 5, (0, 0, 255), -1)
            right_elbow_coordinates.append((right_elbow.x, right_elbow.y))

        out.write(image)

        cv2.imshow('MediaPipe Pose', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
out.release()  # 출력 파일 닫기
cv2.destroyAllWindows()

비디오 파일을 처리하는데 실패했습니다.


In [12]:
# 팔꿈치 좌표 출력
for i, coord in enumerate(right_elbow_coordinates):
    print(f"Frame {i+1}: x = {coord[0] * image_width}, y = {coord[1] * image_height}")

Frame 1: x = 254.1355323791504, y = 301.0883688926697
Frame 2: x = 257.2627830505371, y = 301.9400882720947
Frame 3: x = 253.69329452514648, y = 301.4859366416931
Frame 4: x = 253.7015151977539, y = 298.94646406173706
Frame 5: x = 250.15838623046875, y = 297.7630305290222
Frame 6: x = 248.24954986572266, y = 296.9235849380493
Frame 7: x = 251.4297103881836, y = 296.80488109588623
Frame 8: x = 250.59720993041992, y = 298.73186588287354
Frame 9: x = 248.04737091064453, y = 300.05138397216797
Frame 10: x = 244.66726303100586, y = 296.8107604980469
Frame 11: x = 242.2184944152832, y = 297.11037397384644
Frame 12: x = 246.39276504516602, y = 304.67235803604126
Frame 13: x = 246.53827667236328, y = 307.9030466079712
Frame 14: x = 246.6763687133789, y = 309.79400396347046
Frame 15: x = 255.33714294433594, y = 320.31195402145386
Frame 16: x = 261.39087677001953, y = 335.65605640411377
Frame 17: x = 262.28445053100586, y = 348.776113986969
Frame 18: x = 264.6213722229004, y = 366.24937534332275

In [13]:
# 좌표를 데이터프레임에 저장
import pandas as pd
df = pd.DataFrame(right_elbow_coordinates, columns=["x", "y"])
df.head()

,x,y
0,0.198543,0.418178
1,0.200987,0.419361
2,0.198198,0.418730
3,0.198204,0.415203
4,0.195436,0.413560


In [14]:
output_dir = "D:/Pitch/coordinates"
output_filename = os.path.basename(os.path.splitext(input_file)[0]) + "_elbow_coordinates.csv"
output_file_path = os.path.join(output_dir, output_filename)

df.to_csv(output_file_path, index=False)

OSError: Cannot save file into a non-existent directory: 'D:/Pitch/coordinates'

In [8]:
import cv2

cap = cv2.VideoCapture(output_file)

if (cap.isOpened()== False): 
    print("비디오를 열 수 없습니다.")
  
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        cv2.imshow('Frame', frame)
        # q 키를 눌러 비디오 재생을 중지합니다.
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else: 
        break

cap.release()
cv2.destroyAllWindows()

# q 누르면 비디오 중지됩니다!!! 제발!!!